In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
URL = "https://www.psx.com.pk/psx/resources-and-tools/listings/listed-companies"
COMPANY_URL = "https://www.psx.com.pk/psx/custom-templates/companiesSearch-sector?sector={}&XID={}"

LISTING_CLASS = "notice-update-div"

In [ ]:
response = requests.get(URL)
if not response.status_code == 200:
    raise ValueError(f"Error! Could not fetch page from {URL}.")

page = response.text
soup = BeautifulSoup(page, "lxml")

In [7]:
container = soup.find(class_=LISTING_CLASS)

In [15]:
xid_container = container.find("input", attrs={"id": "XID"})
if not xid_container:
    raise ValueError("Error! Could not find the XID..")

xid = xid_container.get("value")

In [26]:
sector = container.find("select", attrs={"name": "sector"})
if not sector:
    raise ValueError(f"Error! Could not find the sectors section in the site {URL}.")

sector_dict = []
for option in sector.find_all("option"):
    if option.get("value") == "0":
        continue
    option_dict = {"value": option.get("value"), "text": option.text.strip()}
    sector_dict.append(option_dict)